In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

In [2]:
mypath = '../../data/MLgSA/wss/'
mypath1 = '../../data/MLgSA/New_wss/'

onlyfiles = np.asarray([f for f in listdir(mypath) if isfile(join(mypath, f))])
onlyfiles1 = np.asarray([f for f in listdir(mypath1) if isfile(join(mypath1, f))])

onlyfiles.sort()
onlyfiles1.sort()

print(len(onlyfiles), len(onlyfiles1))

short_list = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles])
short_list1 = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles1])

108 30


In [3]:
reader = pv.get_reader(mypath + onlyfiles[0])
mesh  = reader.read()

In [4]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars = 'longitudinal_WSS_@_t=1.1', show_edges=True)
plotter.show()

Widget(value='<iframe src="http://localhost:41071/index.html?ui=P_0x7f9baa06ac70_0&reconnect=auto" class="pyvi…

In [6]:
mesh.point_data['data'] = mesh['longitudinal_WSS_@_t=1.1']

In [70]:
def extract_edges(mesh):
    cells = mesh.n_cells
    cell_types = mesh.celltypes

    edges = set()
    for i  in range(cells):
        if cell_types[i] == 5:  # Assuming triangles
            cell = mesh.get_cell(i)
            edges.add(tuple(sorted(cell.get_edge(0).point_ids)))
            edges.add(tuple(sorted(cell.get_edge(1).point_ids)))
            edges.add(tuple(sorted(cell.get_edge(2).point_ids)))

    edges = np.array(list(edges))
    return edges

In [71]:
edges = extract_edges(mesh)

In [72]:
# Calculate the Laplacian matrix
def compute_laplacian(mesh, edges):
    n_points = mesh.number_of_points

    row_indices = np.hstack([edges[:, 0], edges[:, 1]])
    col_indices = np.hstack([edges[:, 1], edges[:, 0]])
    data = np.ones_like(row_indices)
    
    adjacency_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(n_points, n_points))
    degree_matrix = csr_matrix((np.array(adjacency_matrix.sum(axis=1)).flatten(), (range(n_points), range(n_points))))
    
    laplacian_matrix = degree_matrix - adjacency_matrix
    return laplacian_matrix

laplacian_matrix = compute_laplacian(mesh, edges)

In [74]:
laplacian_matrix = compute_laplacian(mesh, edges)

# Calculate the Laplacian smoothness field (Laplacian applied to the data)
laplacian_field = laplacian_matrix.dot(mesh.point_data['data'])
mesh.point_data['laplacian_field'] = laplacian_field

# Visualize the Laplacian field
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='laplacian_field', cmap='coolwarm', show_edges=True)
plotter.add_scalar_bar('Laplacian Field')
plotter.show()

Widget(value='<iframe src="http://localhost:41071/index.html?ui=P_0x7f9b3e7ee8e0_5&reconnect=auto" class="pyvi…

In [46]:
signal = mesh['laplacian_field']

In [48]:
mesh.get_cell(0).point_ids

array([[ 54.25545165, -18.31217195, -43.54226477],
       [ 54.29228835, -18.29718805, -43.53876179],
       [ 54.27785746, -18.33567043, -43.50361591]])

In [58]:
# Extract edges from the mesh
edges = mesh.extract_all_edges()

# Compute the gradient (difference) at each edge
edge_gradients = []
for edge in edges.lines.reshape((-1, 3))[:, 1:]:
    point1, point2 = edge
    gradient = np.abs(signal[point1] - signal[point2])
    edge_gradients.append(gradient)

edge_gradients = np.array(edge_gradients)
edges.cell_data['Edge Gradient'] = edge_gradients


In [59]:
# Plot the edges colored by the gradient values
plotter = pv.Plotter()
plotter.add_mesh(edges, scalars='Edge Gradient', cmap='viridis')
plotter.add_scalar_bar(title='Edge Gradient')
plotter.show()


Widget(value='<iframe src="http://localhost:41071/index.html?ui=P_0x7f9b30cdda60_3&reconnect=auto" class="pyvi…

In [67]:
np.argmax(edge_gradients)

24755

In [57]:

# Calculate and visualize the first few eigenvectors of the Laplacian
k = 3  # Number of eigenvectors to visualize
eigenvalues, eigenvectors = eigsh(laplacian_matrix, k=k, which='SM')
for i in range(k):
    mesh.point_data[f'eigenvector_{i+1}'] = eigenvectors[:, i]

    plotter = pv.Pplotter()
    plotter.add_mesh(mesh, scalars=f'eigenvector_{i+1}', cmap='viridis', show_edges=True)
    plotter.add_scalar_bar(f'Eigenvector {i+1}')
    plotter.show()

# Visualize Hölder Continuity by highlighting edges with differences scaled by distance
distances = np.linalg.norm(mesh.points[edges[:, 0]] - mesh.points[edges[:, 1]], axis=1)
holder_values = np.abs(data[edges[:, 0]] - data[edges[:, 1]]) / distances ** 0.5
edge_mesh.point_data['holder_values'] = np.repeat(holder_values, 2)

plotter = pv.Plotter()
plotter.add_mesh(edge_mesh, scalars='holder_values', cmap='cool', line_width=2)
plotter.add_scalar_bar('Hölder Continuity Values')
plotter.show()

ValueError: matrix type must be 'f', 'd', 'F', or 'D'